In [ ]:
import time
import numba
import numpy as np
from scipy.sparse import random as sparse_random
from numba import jit
from numba.typed import List
from numba import int64, float64

#import the w500 data
!wget https://transfer.sh/9BSVESkOgv/w500
w_500 = np.loadtxt('/content/w500').astype(np.float64) #import as float for energy calculation
# w_500 = np.loadtxt('w500').astype(np.float64) #import as float for energy calculation

--2024-01-26 14:25:15--  https://transfer.sh/9BSVESkOgv/w500
Resolving transfer.sh (transfer.sh)... 144.76.136.153, 2a01:4f8:200:1097::2
Connecting to transfer.sh (transfer.sh)|144.76.136.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4000500 (3.8M) []
Saving to: ‘w500’

w500                100%[===================>]   3.81M  4.63MB/s    in 0.8s    

2024-01-26 14:25:17 (4.63 MB/s) - ‘w500’ saved [4000500/4000500]



In [ ]:
# compute the sparsity of the w500 matrix
total_non_diagonal_elements = 500 * 500 - 500
non_diagonal_zeros = np.count_nonzero(w_500 == 0) - np.count_nonzero(np.diag(w_500) == 0)
sparsity = (non_diagonal_zeros / total_non_diagonal_elements) * 100
result = f"Sparsity: {sparsity:.2f}%"
result

'Sparsity: 36.73%'

### Find minimal energy state for real-valued x

In [ ]:
def energy(x, w):
    return -0.5 * x.T @ w @ x

n = 7              # system size (nxn)
p = 0.5            # sparsity prob

# enerate sparse symmetric random matrix
w = sparse_random(n, n, density=p, format='coo', data_rvs=np.random.randn).toarray() #create a sparse matrix (and convery to np array)
w = w + w.T            # symmetrize it
np.fill_diagonal(w,0)  # remove diagonal (no self interactions)

# uncomment for ferro-magnetic system
w = (w > 0).astype(int)

print(w)

# uncomment to make it frustrated
# w = (w > 0).astype(int) - (w < 0).astype(int)

# find eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(w)

# minimum eigenvalue of and corresponding eigenvector
min_eigenvalue = np.min(eigenvalues)
min_eigenvector = eigenvectors[:, np.argmin(eigenvalues)]

# normalizing the eigenvector
min_eigenvector_normalized = min_eigenvector / np.linalg.norm(min_eigenvector)

# energy for the min_eigenvector_normalized
energy_min_eigenvector = energy(min_eigenvector_normalized, w)

# Preparing the results for clear presentation
results = {
    "Minimum Eigenvalue": min_eigenvalue,
    "Normalized Eigenvector Corresponding to Minimum Eigenvalue": min_eigenvector_normalized,
    "Energy at Normalized Minimum Eigenvector": energy_min_eigenvector,
}

results

[[0 0 0 1 0 0 0]
 [0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0]
 [1 1 0 0 1 0 0]
 [0 0 0 1 0 1 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]]


{'Minimum Eigenvalue': -1.9318516525781344,
 'Normalized Eigenvector Corresponding to Minimum Eigenvalue': array([ 0.32505758,  0.44403692,  0.        , -0.62796303,  0.44403692,
        -0.22985042, -0.22985042]),
 'Energy at Normalized Minimum Eigenvector': 0.9659258262890685}

# Iterative improvement

### Ferromagnet

In [ ]:
n = 50             # system size (nxn)
p = 0.5            # sparsity prob

# enerate sparse symmetric random matrix
w = sparse_random(n, n, density=p, format='coo', data_rvs=np.random.randn).toarray() #create a sparse matrix (and convery to np array)
w = w + w.T            # symmetrize it
np.fill_diagonal(w,0)  # remove diagonal (no self interactions)

# ferro-magnetic system
w = (w > 0).astype(np.float64)

# number of runs
N_runs = 20
# number of restarts
K_values = [20, 100, 200, 500, 1000, 2000, 4000]
total_Ks = len(K_values)
# number of samples
N_timesteps = 10000

# initialize a list to store the final results
final_results = []

@jit(nopython=True)
def energy(x, w):
    return -0.5 * x.T @ w @ x

@jit(nopython=True)
def run_simulation(K, n, w, N_timesteps):
    lowest_energy = np.inf  # evaluate lowest energy after each k-loop

    for k in range(K):
        x_t = np.random.randint(0, 2, size=n).astype(np.float64) * 2 - 1
        no_improvement_counter = 0          # counter to track early stopping
        current_energy = energy(x_t, w)     # get current energy

        for t in range(N_timesteps):
            index_to_flip = np.random.randint(0, n)
            x_t[index_to_flip] *= -1  # flip the spin
            energy_change = -2 * x_t[index_to_flip] * np.sum(w[index_to_flip] * x_t) # change in energy for single spin flip

            # accept or reject
            if energy_change < 0:
                current_energy += energy_change  # update current energy if accepted
            else:
                x_t[index_to_flip] *= -1         # flip the spin back if rejected

        if current_energy < lowest_energy:       # update lowest energy if k-loop gave a lower one
            lowest_energy = current_energy

    return lowest_energy

# warm-up run to trigger JIT compilation and other initializations (proper cpu time tracking)
_ = run_simulation(K_values[0], n, w, 100)

# main loop over K_values
for i, K in enumerate(K_values):
    energies = np.zeros(N_runs, dtype=np.float64)  # store energies for each K run
    total_cpu_time = 0  # Initialize total CPU time for each K

    # loop over different runs
    for run in range(N_runs):
        start_time = time.time()  # track time for a single k-loop

        # run the simulation K times and store the lowest energy
        energies[run] = run_simulation(K, n, w, N_timesteps)

        cpu_time = time.time() - start_time
        total_cpu_time += cpu_time
        print(f"Run {run+1}/{N_runs} for K={K} completed in {cpu_time:.2f} seconds.")

    avg_cpu_time = total_cpu_time / N_runs  # calculate average CPU time
    avg_energy = np.mean(energies)
    std_deviation = np.std(energies)

    # Update progress after each K
    print(f"Completed K = {K} ({i + 1} of {total_Ks})")
    final_results.append([K, avg_cpu_time, avg_energy, std_deviation])


print('Done! \n ----------------------------------------------------------------------------------------------------')

# Print the table (LaTeX format)
print("\\begin{tabular}{ccc}")
print("\\toprule")
print("K & CPU Time (sec) & E \\\\")  # Header row
print("\\midrule")
for row in final_results:
    print(f"{row[0]} & {row[1]:.2f} & {row[2]:.0f} $\pm$ {row[3]:.0f} \\\\")  # Data rows
print("\\bottomrule")
print("\\end{tabular}")

Run 1/20 for K=20 completed in 0.11 seconds.
Run 2/20 for K=20 completed in 0.12 seconds.
Run 3/20 for K=20 completed in 0.08 seconds.
Run 4/20 for K=20 completed in 0.05 seconds.
Run 5/20 for K=20 completed in 0.08 seconds.
Run 6/20 for K=20 completed in 0.08 seconds.
Run 7/20 for K=20 completed in 0.10 seconds.
Run 8/20 for K=20 completed in 0.11 seconds.
Run 9/20 for K=20 completed in 0.10 seconds.
Run 10/20 for K=20 completed in 0.10 seconds.
Run 11/20 for K=20 completed in 0.12 seconds.
Run 12/20 for K=20 completed in 0.11 seconds.
Run 13/20 for K=20 completed in 0.10 seconds.
Run 14/20 for K=20 completed in 0.15 seconds.
Run 15/20 for K=20 completed in 0.19 seconds.
Run 16/20 for K=20 completed in 0.22 seconds.
Run 17/20 for K=20 completed in 0.18 seconds.
Run 18/20 for K=20 completed in 0.17 seconds.
Run 19/20 for K=20 completed in 0.14 seconds.
Run 20/20 for K=20 completed in 0.12 seconds.
Completed K = 20 (1 of 7)
Run 1/20 for K=100 completed in 0.70 seconds.
Run 2/20 for K=10

### Frustrated (Random)

In [ ]:
n = 50             # system size (nxn)
p = 0.5            # sparsity prob

# enerate sparse symmetric random matrix
w = sparse_random(n, n, density=p, format='coo', data_rvs=np.random.randn).toarray() #create a sparse matrix (and convery to np array)
w = w + w.T            # symmetrize it
np.fill_diagonal(w,0)  # remove diagonal (no self interactions)

# frustrated system
w = (w > 0).astype(int) - (w < 0).astype(np.float64)

# number of runs
N_runs = 20
# number of restarts
K_values = [20, 100, 200, 500, 1000, 2000, 4000]
total_Ks = len(K_values)
# number of samples
N_timesteps = 10000

# initialize a list to store the final results
final_results = []

@jit(nopython=True)
def energy(x, w):
    return -0.5 * x.T @ w @ x

@jit(nopython=True)
def run_simulation(K, n, w, N_timesteps):
    lowest_energy = np.inf  # evaluate lowest energy after each k-loop

    for k in range(K):
        x_t = np.random.randint(0, 2, size=n).astype(np.float64) * 2 - 1
        no_improvement_counter = 0          # counter to track early stopping
        current_energy = energy(x_t, w)     # get current energy

        for t in range(N_timesteps):
            index_to_flip = np.random.randint(0, n)
            x_t[index_to_flip] *= -1  # flip the spin
            energy_change = -2 * x_t[index_to_flip] * np.sum(w[index_to_flip] * x_t) # change in energy for single spin flip

            # accept or reject according to iterative improvement scheme
            if energy_change < 0:
                current_energy += energy_change  # update current energy if accepted
            else:
                x_t[index_to_flip] *= -1         # flip the spin back if rejected

        if current_energy < lowest_energy:       # update lowest energy if k-loop gave a lower one
            lowest_energy = current_energy

    return lowest_energy

# warm-up run to trigger JIT compilation and other initializations (proper cpu time tracking)
_ = run_simulation(K_values[0], n, w, 100)

# main loop over K_values
for i, K in enumerate(K_values):
    energies = np.zeros(N_runs, dtype=np.float64)  # store energies for each K run
    total_cpu_time = 0  # Initialize total CPU time for each K

    # loop over different runs
    for run in range(N_runs):
        start_time = time.time()  # track time for a single k-loop

        # run the simulation K times and store the lowest energy
        energies[run] = run_simulation(K, n, w, N_timesteps)

        cpu_time = time.time() - start_time
        total_cpu_time += cpu_time
        print(f"Run {run+1}/{N_runs} for K={K} completed in {cpu_time:.2f} seconds.")

    avg_cpu_time = total_cpu_time / N_runs  # calculate average CPU time
    avg_energy = np.mean(energies)
    std_deviation = np.std(energies)

    # Update progress after each K
    print(f"Completed K = {K} ({i + 1} of {total_Ks})")
    final_results.append([K, avg_cpu_time, avg_energy, std_deviation])


print('Done! \n ----------------------------------------------------------------------------------------------------')
# Print the table (LaTeX format)
print("\\begin{tabular}{ccc}")
print("\\toprule")
print("K & CPU Time (sec) & E \\\\")  # Header row
print("\\midrule")
for row in final_results:
    print(f"{row[0]} & {row[1]:.2f} & {row[2]:.0f} $\pm$ {row[3]:.0f} \\\\")  # Data rows
print("\\bottomrule")
print("\\end{tabular}")

Run 1/20 for K=20 completed in 0.03 seconds.
Run 2/20 for K=20 completed in 0.03 seconds.
Run 3/20 for K=20 completed in 0.04 seconds.
Run 4/20 for K=20 completed in 0.03 seconds.
Run 5/20 for K=20 completed in 0.03 seconds.
Run 6/20 for K=20 completed in 0.03 seconds.
Run 7/20 for K=20 completed in 0.03 seconds.
Run 8/20 for K=20 completed in 0.03 seconds.
Run 9/20 for K=20 completed in 0.03 seconds.
Run 10/20 for K=20 completed in 0.03 seconds.
Run 11/20 for K=20 completed in 0.03 seconds.
Run 12/20 for K=20 completed in 0.03 seconds.
Run 13/20 for K=20 completed in 0.03 seconds.
Run 14/20 for K=20 completed in 0.03 seconds.
Run 15/20 for K=20 completed in 0.03 seconds.
Run 16/20 for K=20 completed in 0.03 seconds.
Run 17/20 for K=20 completed in 0.03 seconds.
Run 18/20 for K=20 completed in 0.04 seconds.
Run 19/20 for K=20 completed in 0.03 seconds.
Run 20/20 for K=20 completed in 0.04 seconds.
Completed K = 20 (1 of 7)
Run 1/20 for K=100 completed in 0.16 seconds.
Run 2/20 for K=10

### Frustrated (W500)

In [ ]:
# number of spins
n = 500
# number of runs
N_runs = 20
# number of restarts
K_values = [20, 100, 200, 500, 1000, 2000, 4000]
total_Ks = len(K_values)
# number of samples
N_timesteps = 3500

# initialize a list to store the final results
final_results = []

@jit(nopython=True)
def energy(x, w):
    return -0.5 * x.T @ w @ x

@jit(nopython=True)
def run_simulation(K, n, w_500, N_timesteps):
    lowest_energy = np.inf  #evaluate lowest energy after each k-loop

    for k in range(K):
        x_t = np.random.randint(0, 2, size=n).astype(np.float64) * 2 - 1
        no_improvement_counter = 0          # counter to track early stopping
        current_energy = energy(x_t, w_500) # get current energy, calculate once (instead of a every iteration of the loop)

        for t in range(N_timesteps):
            index_to_flip = np.random.randint(0, n)
            x_t[index_to_flip] *= -1  # flip the spin
            energy_change = -2 * x_t[index_to_flip] * np.sum(w_500[index_to_flip] * x_t) #change in energy for single spin flip

            #accept
            if energy_change < 0:
                current_energy += energy_change  # update current energy if accepted
            #reject
            else:
                x_t[index_to_flip] *= -1         # flip the spin back to orignal state if rejected

        if current_energy < lowest_energy:       #update lowest energy if k-loop gave a lower one
            lowest_energy = current_energy

    return lowest_energy

# warm-up run to trigger JIT compilation and other initializations (proper cpu time tracking)
_ = run_simulation(K_values[0], n, w_500, 100)

# main loop over K_values
for i, K in enumerate(K_values):
    energies = np.zeros(N_runs, dtype=np.float64)  # store energies for each K run
    total_cpu_time = 0  # Initialize total CPU time for each K

    # loop over different runs
    for run in range(N_runs):
        start_time = time.time()  # track time for a single k-loop

        # run the simulation K times and store the lowest energy
        energies[run] = run_simulation(K, n, w_500, N_timesteps)

        cpu_time = time.time() - start_time
        total_cpu_time += cpu_time
        print(f"Run {run+1}/{N_runs} for K={K} completed in {cpu_time:.2f} seconds.")

    avg_cpu_time = total_cpu_time / N_runs  # calculate average CPU time
    avg_energy = np.mean(energies)
    std_deviation = np.std(energies)

    # Update progress after each K
    print(f"Completed K = {K} ({i + 1} of {total_Ks})")
    final_results.append([K, avg_cpu_time, avg_energy, std_deviation])


print('Done! \n ----------------------------------------------------------------------------------------------------')

# Print the table (LaTeX format)
print("\\begin{tabular}{ccc}")
print("\\toprule")
print("K & CPU Time (sec) & E \\\\")  # Header row
print("\\midrule")
for row in final_results:
    print(f"{row[0]} & {row[1]:.2f} & {row[2]:.0f} $\pm$ {row[3]:.0f} \\\\")  # Data rows
print("\\bottomrule")
print("\\end{tabular}")

Run 1/20 for K=20 completed in 0.14 seconds.
Run 2/20 for K=20 completed in 0.20 seconds.
Run 3/20 for K=20 completed in 0.16 seconds.
Run 4/20 for K=20 completed in 0.18 seconds.
Run 5/20 for K=20 completed in 0.19 seconds.
Run 6/20 for K=20 completed in 0.14 seconds.
Run 7/20 for K=20 completed in 0.11 seconds.
Run 8/20 for K=20 completed in 0.09 seconds.
Run 9/20 for K=20 completed in 0.10 seconds.
Run 10/20 for K=20 completed in 0.09 seconds.
Run 11/20 for K=20 completed in 0.09 seconds.
Run 12/20 for K=20 completed in 0.09 seconds.
Run 13/20 for K=20 completed in 0.09 seconds.
Run 14/20 for K=20 completed in 0.09 seconds.
Run 15/20 for K=20 completed in 0.09 seconds.
Run 16/20 for K=20 completed in 0.09 seconds.
Run 17/20 for K=20 completed in 0.09 seconds.
Run 18/20 for K=20 completed in 0.09 seconds.
Run 19/20 for K=20 completed in 0.08 seconds.
Run 20/20 for K=20 completed in 0.09 seconds.
Completed K = 20 (1 of 7)
Run 1/20 for K=100 completed in 0.79 seconds.
Run 2/20 for K=10

# Simulated annealing


## Exponential schedule

### With fixed amount of samples

In [ ]:
@jit(nopython=True)
def energy(x, w):
    return -0.5 * x.T @ w @ x

@jit(nopython=True)
def estimate_initial_temperature(x_t, n=500):
    energy_change_max = 0
    beta = 0.1 #this heavily influences results. What is "high" temperature? Getting good results now though
    current_energy = energy(x_t, w_500) # get current energy times beta, calculate once (instead of every iteration of the loop)

    for _ in range(50000):  # Main simulation loop
          index_to_flip = np.random.randint(0, n)
          x_t[index_to_flip] *= -1  # flip the spin
          energy_change = -2 * x_t[index_to_flip] * np.sum(w_500[index_to_flip] * x_t) # change in energy for single spin flip w/ beta

          #accept
          if energy_change < 0 or np.random.rand() < np.exp(-energy_change * beta):
              current_energy += energy_change  # update current energy if accepted
          # reject
          else:
              x_t[index_to_flip] *= -1         # flip the spin back to original state if rejected

          #store largest energy diff
          if energy_change > energy_change_max:
            energy_change_max = energy_change

    initial_temp = energy_change_max #the maximum energy change during this sampling is the initial temperature
    return initial_temp

@jit(nopython=True)
def run_simulation_exponential_annealing(f, L, n, w_500, N_timesteps):
    lowest_energy = np.inf  # evaluate lowest energy after each k-loop

    x_t = np.random.randint(0, 2, size=n).astype(np.float64) * 2 - 1
    initial_temp = estimate_initial_temperature(x_t) # get the initial temperature
    beta = 1/initial_temp # get initial beta value

    current_energy = energy(x_t, w_500) # get current energy, calculate once (instead of every iteration of the loop)

    for t in range(N_timesteps):  # Main simulation loop
        previous_energy = lowest_energy

        index_to_flip = np.random.randint(0, n)
        x_t[index_to_flip] *= -1  # flip the spin
        energy_change = -2 * x_t[index_to_flip] * np.sum(w_500[index_to_flip] * x_t) # change in energy for single spin flip w/ beta

        #accept
        if energy_change < 0 or np.random.rand() < np.exp(-energy_change * beta):
            current_energy += energy_change  # update current energy if accepted
        # reject
        else:
            x_t[index_to_flip] *= -1         # flip the spin back to original state if rejected

        if current_energy < lowest_energy:       # update lowest energy if we got one gave a lower one
            lowest_energy = current_energy

        if t % L == 0:  # update temperature according to exponential schedule
            beta *= f

    return lowest_energy

x_t = np.random.randint(0, 2, size=500).astype(np.float64) * 2 - 1
initial_temp = estimate_initial_temperature(x_t)
print('Initial estimation of temperature is', initial_temp)

Initial estimation of temperature is 194.0


In [ ]:
# number of spins
n = 500
# number of runs
N_runs = 20

N_timesteps = 1000000
# f_values
f_values = [1.01, 1.001, 1.002, 1.0002]
# # number of samples per beta value [length of chain]
L_values = [500, 500, 500, 1000]

# initialize a list to store the final results
final_results = []

np.random.seed(42)

# warm-up run to trigger JIT compilation and other initializations (proper cpu time tracking)
_ = run_simulation_exponential_annealing(f_values[0], L_values[0], n, w_500, 100)

# main loop over f
for i, f in enumerate(f_values):
    L = L_values[i]
    energies = np.zeros(N_runs, dtype=np.float64)  # store energies for each K run
    total_cpu_time = 0  # Initialize total CPU time for each K

    # loop over different runs
    for run in range(N_runs):
        start_time = time.time()  # track time for a single k-loop

        # run the simulation K times and store the lowest energy
        energies[run] = run_simulation_exponential_annealing(f, L, n, w_500, N_timesteps)

        cpu_time = time.time() - start_time
        total_cpu_time += cpu_time
        print(f"Run {run+1}/{N_runs} for f={f} completed in {cpu_time:.2f} seconds.")

    avg_cpu_time = total_cpu_time / N_runs  # calculate average CPU time
    avg_energy = np.mean(energies)
    std_deviation = np.std(energies)

    # Update progress after each K
    print(f"Completed round for f = {f} and L = {L}")
    final_results.append([f, L, avg_cpu_time, avg_energy, std_deviation])


print('Done! \n ----------------------------------------------------------------------------------------------------')

# Print the table (LaTeX format)
print("\\begin{tabular}{cccc}")
print("\\toprule")
print("$\Delta \beta$ & L & CPU Time (sec) & E \\\\")  # Header row
print("\\midrule")
for row in final_results:
    print(f"{row[0]} & {row[1]:.2f} & {row[2]:.2f} & {row[3]:.0f} $\pm$ {row[4]:.0f} \\\\")  # Data rows
print("\\bottomrule")
print("\\end{tabular}")

Run 1/20 for f=1.01 completed in 1.46 seconds.
Run 2/20 for f=1.01 completed in 1.43 seconds.
Run 3/20 for f=1.01 completed in 1.43 seconds.
Run 4/20 for f=1.01 completed in 1.76 seconds.
Run 5/20 for f=1.01 completed in 2.28 seconds.
Run 6/20 for f=1.01 completed in 1.63 seconds.
Run 7/20 for f=1.01 completed in 1.41 seconds.
Run 8/20 for f=1.01 completed in 1.80 seconds.
Run 9/20 for f=1.01 completed in 2.96 seconds.
Run 10/20 for f=1.01 completed in 3.06 seconds.
Run 11/20 for f=1.01 completed in 1.30 seconds.
Run 12/20 for f=1.01 completed in 1.30 seconds.
Run 13/20 for f=1.01 completed in 1.32 seconds.
Run 14/20 for f=1.01 completed in 1.34 seconds.
Run 15/20 for f=1.01 completed in 1.31 seconds.
Run 16/20 for f=1.01 completed in 1.30 seconds.
Run 17/20 for f=1.01 completed in 1.30 seconds.
Run 18/20 for f=1.01 completed in 1.56 seconds.
Run 19/20 for f=1.01 completed in 1.51 seconds.
Run 20/20 for f=1.01 completed in 1.49 seconds.
Completed round for f = 1.01 and L = 500
Run 1/20

### With variance convergence

Bert appears to use no fixed sample size, but some other measure of convergence indicated by the different CPU times. Let's also try to do that by using the energy variance = 0 as a measure of convergence

In [ ]:
# @jit(nopython=True)
def run_simulation_exponential_annealing(f, L, n, w_500, N_timesteps, early_stop_threshold=10000000):
    lowest_energy = np.inf
    energy_variance_k = np.inf  # initial variance set to infinity
    energies = []  # list to keep track of the energy at each step

    x_t = np.random.randint(0, 2, size=n).astype(np.float64) * 2 - 1
    initial_temp = estimate_initial_temperature(x_t)  # get the initial temperature
    beta = 1 / initial_temp  # get initial beta value

    current_energy = energy(x_t, w_500)  # get current energy
    energies.append(current_energy)  # Keep track of the current energy

    t = 0
    while energy_variance_k > 0 and t < N_timesteps: #keep timesteps for the warmup
        previous_energy = current_energy

        index_to_flip = np.random.randint(0, n)
        x_t[index_to_flip] *= -1  # flip the spin
        energy_change = -2 * x_t[index_to_flip] * np.sum(w_500[index_to_flip] * x_t)  # change in energy for single spin flip w/ beta

        # accept
        if energy_change < 0 or np.random.rand() < np.exp(-energy_change * beta):
            current_energy += energy_change  # update current energy if accepted
            energies.append(current_energy)
        # reject
        else:
            x_t[index_to_flip] *= -1  # flip the spin back to original state if rejected
            energies.append(current_energy)

        if current_energy < lowest_energy:  # update lowest energy if we got one gave a lower one
            lowest_energy = current_energy

        if t % L == 0 and len(energies) >= L:  # update temperature according to aarts schedule
            # compute the variance of the last L samples
            energy_variance_k = np.var(energies[-L:], ddof=1)  # ddof=1 provides an unbiased estimator

            beta *= f

    return lowest_energy


In [ ]:
# number of spins
n = 500
# number of runs
N_runs = 20

# f_values
f_values = [1.01, 1.001, 1.002, 1.0002]
# # number of samples per beta value [length of chain]
L_values = [500, 500, 500, 1000]

# initialize a list to store the final results
final_results = []

np.random.seed(42)

# warm-up run to trigger JIT compilation and other initializations (proper cpu time tracking)
_ = run_simulation_exponential_annealing(f_values[0], L_values[0], n, w_500, 100)

# main loop over f
for i, f in enumerate(f_values):
    L = L_values[i]
    energies = np.zeros(N_runs, dtype=np.float64)  # store energies for each K run
    total_cpu_time = 0  # Initialize total CPU time for each K

    # loop over different runs
    for run in range(N_runs):
        start_time = time.time()  # track time for a single k-loop

        # run the simulation K times and store the lowest energy
        energies[run] = run_simulation_exponential_annealing(f, L, n, w_500, N_timesteps)

        cpu_time = time.time() - start_time
        total_cpu_time += cpu_time
        print(f"Run {run+1}/{N_runs} for f={f} completed in {cpu_time:.2f} seconds.")

    avg_cpu_time = total_cpu_time / N_runs  # calculate average CPU time
    avg_energy = np.mean(energies)
    std_deviation = np.std(energies)

    # Update progress after each K
    print(f"Completed round for f = {f} and L = {L}")
    final_results.append([f, L, avg_cpu_time, avg_energy, std_deviation])


print('Done! \n ----------------------------------------------------------------------------------------------------')

# Print the table (LaTeX format)
print("\\begin{tabular}{cccc}")
print("\\toprule")
print("$\Delta  \beta$ & L & CPU Time (sec) & E \\\\")  # Header row
print("\\midrule")
for row in final_results:
    print(f"{row[0]} & {row[1]:.2f} & {row[2]:.2f} & {row[3]:.0f} $\pm$ {row[4]:.0f} \\\\")  # Data rows
print("\\bottomrule")
print("\\end{tabular}")

Run 1/20 for f=1.01 completed in 0.54 seconds.
Run 2/20 for f=1.01 completed in 0.44 seconds.
Run 3/20 for f=1.01 completed in 0.37 seconds.
Run 4/20 for f=1.01 completed in 0.36 seconds.
Run 5/20 for f=1.01 completed in 0.48 seconds.
Run 6/20 for f=1.01 completed in 0.39 seconds.
Run 7/20 for f=1.01 completed in 0.35 seconds.
Run 8/20 for f=1.01 completed in 0.40 seconds.
Run 9/20 for f=1.01 completed in 0.37 seconds.
Run 10/20 for f=1.01 completed in 0.33 seconds.
Run 11/20 for f=1.01 completed in 0.39 seconds.
Run 12/20 for f=1.01 completed in 0.42 seconds.
Run 13/20 for f=1.01 completed in 0.32 seconds.
Run 14/20 for f=1.01 completed in 0.25 seconds.
Run 15/20 for f=1.01 completed in 0.26 seconds.
Run 16/20 for f=1.01 completed in 0.34 seconds.
Run 17/20 for f=1.01 completed in 0.37 seconds.
Run 18/20 for f=1.01 completed in 0.46 seconds.
Run 19/20 for f=1.01 completed in 0.48 seconds.
Run 20/20 for f=1.01 completed in 0.37 seconds.
Completed round for f = 1.01 and L = 500
Run 1/20

### With accepted amount of samples

That appears to not do the trick, we can also try to have a fixed amount of *accepted* samples. However, this does not work because for large temperatures we accept barely any samples

In [ ]:
@jit(nopython=True)
def energy(x, w):
    return -0.5 * x.T @ w @ x

@jit(nopython=True)
def estimate_initial_temperature(x_t, n=500):
    energy_change_max = 0
    beta = 0.1 #this heavily influences results. What is "high" temperature? Getting good results now though
    current_energy = energy(x_t, w_500) # get current energy times beta, calculate once (instead of every iteration of the loop)

    for _ in range(50000):  # Main simulation loop
          index_to_flip = np.random.randint(0, n)
          x_t[index_to_flip] *= -1  # flip the spin
          energy_change = -2 * x_t[index_to_flip] * np.sum(w_500[index_to_flip] * x_t) # change in energy for single spin flip w/ beta

          #accept
          if energy_change < 0 or np.random.rand() < np.exp(-energy_change * beta):
              current_energy += energy_change  # update current energy if accepted
          # reject
          else:
              x_t[index_to_flip] *= -1         # flip the spin back to original state if rejected

          #store largest energy diff
          if energy_change > energy_change_max:
            energy_change_max = energy_change

    initial_temp = energy_change_max #the maximum energy change during this sampling is the initial temperature
    return initial_temp

@jit(nopython=True)
def run_simulation_exponential_annealing(f, L, n, w_500, accepted_samples_threshold):
    lowest_energy = np.inf  # evaluate lowest energy after each k-loop
    accepted_samples_count = 0  # counter for accepted samples

    x_t = np.random.randint(0, 2, size=n).astype(np.float64) * 2 - 1
    initial_temp = estimate_initial_temperature(x_t)  # get the initial temperature
    beta = 1 / initial_temp  # get initial beta value

    current_energy = energy(x_t, w_500)  # get current energy, calculate once

    while accepted_samples_count < accepted_samples_threshold:  # modified loop condition
        index_to_flip = np.random.randint(0, n)
        x_t[index_to_flip] *= -1  # flip the spin
        energy_change = -2 * x_t[index_to_flip] * np.sum(w_500[index_to_flip] * x_t)

        # accept
        if energy_change < 0 or np.random.rand() < np.exp(-energy_change * beta):
            current_energy += energy_change  # update current energy if accepted
            accepted_samples_count += 1  # increment counter
        # reject
        else:
            x_t[index_to_flip] *= -1  # flip the spin back to original state if rejected

        if current_energy < lowest_energy:  # update lowest energy if we got one that's lower
            lowest_energy = current_energy

        if accepted_samples_count % L == 0:  # update temperature according to exponential schedule
            beta *= f

    return lowest_energy



In [ ]:
# number of spins
n = 500
# number of runs
N_runs = 20

accepted_samples_threshold = 1000  # set your desired threshold
# f_values
f_values = [1.01, 1.001, 1.002, 1.0002]
# # number of samples per beta value [length of chain]
L_values = [500, 500, 500, 1000]

# initialize a list to store the final results
final_results = []


# warm-up run to trigger JIT compilation and other initializations (proper cpu time tracking)
_ = run_simulation_exponential_annealing(f_values[0], L_values[0], n, w_500, 100)

# main loop over f
for i, f in enumerate(f_values):
    L = L_values[i]
    energies = np.zeros(N_runs, dtype=np.float64)  # store energies for each K run
    total_cpu_time = 0  # Initialize total CPU time for each K

    # loop over different runs
    for run in range(N_runs):
        start_time = time.time()  # track time for a single k-loop

        # run the simulation K times and store the lowest energy
        energies[run] = run_simulation_exponential_annealing(f, L, n, w_500, N_timesteps)

        cpu_time = time.time() - start_time
        total_cpu_time += cpu_time
        print(f"Run {run+1}/{N_runs} for f={f} completed in {cpu_time:.2f} seconds.")

    avg_cpu_time = total_cpu_time / N_runs  # calculate average CPU time
    avg_energy = np.mean(energies)
    std_deviation = np.std(energies)

    # Update progress after each K
    print(f"Completed round for f = {f} and L = {L}")
    final_results.append([f, L, avg_cpu_time, avg_energy, std_deviation])


print('Done! \n ----------------------------------------------------------------------------------------------------')

# Print the table (LaTeX format)
print("\\begin{tabular}{cccc}")
print("\\toprule")
print("$\Delta \beta$ & L & CPU Time (sec) & E \\\\")  # Header row
print("\\midrule")
for row in final_results:
    print(f"{row[0]} & {row[1]:.2f} & {row[2]:.2f} & {row[3]:.0f} $\pm$ {row[4]:.0f} \\\\")  # Data rows
print("\\bottomrule")
print("\\end{tabular}")

## Aarts

In [ ]:
# @jit(nopython=True)
def run_simulation_aarts_annealing(delta_beta,L, n, w_500, N_timesteps):
    lowest_energy = np.inf  # evaluate lowest energy after each k-loop
    energy_variance_k = np.inf  # Initial variance set to infinity

    energies = []  # List to keep track of the energy at each step
    x_t = np.random.randint(0, 2, size=n).astype(np.float64) * 2 - 1
    initial_temp = estimate_initial_temperature(x_t)  # get the initial temperature
    beta = 1 / initial_temp  # get initial beta value

    current_energy = energy(x_t, w_500)  # get current energy
    energies.append(current_energy)  # Keep track of the current energy


    t = 0
    while energy_variance_k > 0 and t < N_timesteps:
        index_to_flip = np.random.randint(0, n)
        x_t[index_to_flip] *= -1  # flip the spin
        energy_change = -2 * x_t[index_to_flip] * np.sum(w_500[index_to_flip] * x_t)  # change in energy for single spin flip

        # accept
        if energy_change < 0 or np.random.rand() < np.exp(-energy_change * beta):
            current_energy += energy_change  # update current energy if accepted
            energies.append(current_energy)  # Keep track of the new energy
        # reject
        else:
            x_t[index_to_flip] *= -1  # flip the spin back to original state if rejected
            energies.append(current_energy)  # Energy stays the same, but we still record it

        if current_energy < lowest_energy:  # update lowest energy if we got one that gave a lower one
            lowest_energy = current_energy

        if t % L == 0 and len(energies) >= L:  # update temperature according to aarts schedule
            # compute the variance of the last L samples
            energy_variance_k = np.var(energies[-L:], ddof=1)  # ddof=1 provides an unbiased estimator

            beta += delta_beta / np.sqrt(energy_variance_k)

        t += 1

    return lowest_energy

In [ ]:
# number of spins
n = 500
# number of runs
N_runs = 20
# Δβ values to iterate over
delta_betas = [0.1, 0.01, 0.001, 0.0001]
# L value for each Δβ
L_values = [500, 500, 500, 1000]
# number of samples (timesteps)
N_timesteps = 100000

# initialize a list to store the final results
final_results = []

# warm-up run to trigger JIT compilation and other initializations (proper cpu time tracking)
_ = run_simulation_aarts_annealing(delta_betas[0], L_values[0], n, w_500, 100)

# main loop over Δβ values
for delta_beta, L in zip(delta_betas, L_values):
    energies = np.zeros(N_runs, dtype=np.float64)  # store energies for each run
    total_cpu_time = 0  # Initialize total CPU time for each Δβ

    # loop over different runs
    for run in range(N_runs):
        start_time = time.time()  # track time for a single run

        # run the simulation and store the lowest energy
        energies[run] = run_simulation_aarts_annealing(delta_beta, L, n, w_500, N_timesteps)

        cpu_time = time.time() - start_time
        total_cpu_time += cpu_time
        print(f"Run {run+1}/{N_runs} for Δβ={delta_beta} completed in {cpu_time:.2f} seconds.")

    avg_cpu_time = total_cpu_time / N_runs  # calculate average CPU time
    avg_energy = np.mean(energies)
    std_deviation = np.std(energies)

    # Update progress after each Δβ
    print(f"Completed Δβ = {delta_beta}")
    final_results.append([delta_beta, L, avg_cpu_time, avg_energy, std_deviation])

print('Done! \n ----------------------------------------------------------------------------------------------------')

# Print the table (LaTeX format)
print("\\begin{tabular}{ccccc}")
print("\\toprule")
print("Δβ & L & CPU (sec) & E \\\\")  # Header row
print("\\midrule")
for row in final_results:
    print(f"{row[0]} & {row[1]} & {row[2]:.2f} & {row[3]:.0f} $\pm$ {row[4]:.0f} \\\\")  # Data rows
print("\\bottomrule")
print("\\end{tabular}")

<ipython-input-91-d8d7460f5854>:35: RuntimeWarning: divide by zero encountered in double_scalars
  beta += delta_beta / np.sqrt(energy_variance_k)


Run 1/20 for Δβ=0.1 completed in 3.63 seconds.
Run 2/20 for Δβ=0.1 completed in 1.31 seconds.
Run 3/20 for Δβ=0.1 completed in 1.17 seconds.
Run 4/20 for Δβ=0.1 completed in 1.20 seconds.
Run 5/20 for Δβ=0.1 completed in 1.16 seconds.
Run 6/20 for Δβ=0.1 completed in 1.25 seconds.
Run 7/20 for Δβ=0.1 completed in 1.28 seconds.
Run 8/20 for Δβ=0.1 completed in 1.33 seconds.
Run 9/20 for Δβ=0.1 completed in 1.17 seconds.
Run 10/20 for Δβ=0.1 completed in 1.77 seconds.
Run 11/20 for Δβ=0.1 completed in 1.51 seconds.
Run 12/20 for Δβ=0.1 completed in 1.27 seconds.
Run 13/20 for Δβ=0.1 completed in 1.28 seconds.
Run 14/20 for Δβ=0.1 completed in 1.15 seconds.
Run 15/20 for Δβ=0.1 completed in 1.16 seconds.
Run 16/20 for Δβ=0.1 completed in 1.24 seconds.
Run 17/20 for Δβ=0.1 completed in 1.23 seconds.
Run 18/20 for Δβ=0.1 completed in 1.24 seconds.
Run 19/20 for Δβ=0.1 completed in 1.52 seconds.
Run 20/20 for Δβ=0.1 completed in 1.71 seconds.
Completed Δβ = 0.1
Run 1/20 for Δβ=0.01 completed

KeyboardInterrupt: 